**7.3. Build AI Agent with Microsoft's AutoGen**

In [4]:
!pip install -q -U "autogen-agentchat" "autogen-ext[openai]"


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [5]:
import os
import asyncio

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [6]:
# openai api key for llm call
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"

# openweathermap api key
OPENWEATHER_API_KEY = "your_open_weather_map_api_key"

In [7]:
# 1. Define a model client (OpenAI in this case)
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="YOUR_OPENAI_API_KEY",  # Use env variable or directly set
)

In [8]:
# 2. Define weather retrieval tool function
async def get_weather(city: str) -> str:
    """
    Fetch the current weather from OpenWeatherMap API for a given city.
    """
    try:
        url = "https://api.openweathermap.org/data/2.5/weather"
        params = {
            "q": city,
            "appid": OPENWEATHER_API_KEY,
            "units": "metric",
        }
        response = requests.get(url, params=params)
        data = response.json()

        if response.status_code != 200 or "weather" not in data:
            return f"⚠️ Could not fetch weather for '{city}'."

        desc = data["weather"][0]["description"].capitalize()
        temp = data["main"]["temp"]
        location = data["name"]
        return f"📍 {location}: {desc}, {temp}°C"
    
    except Exception as e:
        return f"❌ Error fetching weather: {str(e)}"

In [9]:
# 3. Define the AssistantAgent with the tool and streaming
agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message=(
        "You are a helpful weather assistant. If the user asks about weather, "
        "use the 'get_weather' tool to find real-time information."
    ),
    reflect_on_tool_use=True,
    model_client_stream=True,
)

In [10]:
# 4. Main entrypoint (use Console to display response stream)
async def main():
    await Console(agent.run_stream(task="What is the weather in Chennai?"))
    await model_client.close()

# 5. Run the async main function
await main()

---------- TextMessage (user) ----------
What is the weather in Chennai?
---------- ToolCallRequestEvent (weather_agent) ----------
[FunctionCall(id='call_FqpRtHmbFdZhXvkR7ZGi5yi1', arguments='{"city":"Chennai"}', name='get_weather')]
---------- ToolCallExecutionEvent (weather_agent) ----------
[FunctionExecutionResult(content='📍 Chennai: Scattered clouds, 30.54°C', name='get_weather', call_id='call_FqpRtHmbFdZhXvkR7ZGi5yi1', is_error=False)]
---------- ModelClientStreamingChunkEvent (weather_agent) ----------
The current weather in Chennai is scattered clouds with a temperature of 30.54°C.


**Complete Code**

In [12]:
import os
import asyncio
import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Optional: Use .env or set your OpenWeatherMap API key here 
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"
OPENWEATHER_API_KEY = "your_open_weather_map_api_key"

# 1. Define a model client (OpenAI in this case)
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="YOUR_OPENAI_API_KEY",  # Use env variable or directly set
)

# 2. Define weather retrieval tool function
async def get_weather(city: str) -> str:
    """
    Fetch the current weather from OpenWeatherMap API for a given city.
    """
    try:
        url = "https://api.openweathermap.org/data/2.5/weather"
        params = {
            "q": city,
            "appid": OPENWEATHER_API_KEY,
            "units": "metric",
        }
        response = requests.get(url, params=params)
        data = response.json()

        if response.status_code != 200 or "weather" not in data:
            return f"⚠️ Could not fetch weather for '{city}'."

        desc = data["weather"][0]["description"].capitalize()
        temp = data["main"]["temp"]
        location = data["name"]
        return f"📍 {location}: {desc}, {temp}°C"
    
    except Exception as e:
        return f"❌ Error fetching weather: {str(e)}"


# 3. Define the AssistantAgent with the tool and streaming
agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message=(
        "You are a helpful weather assistant. If the user asks about weather, "
        "use the 'get_weather' tool to find real-time information."
    ),
    reflect_on_tool_use=True,
    model_client_stream=True,
)

# 4. Main entrypoint (use Console to display response stream)
async def main():
    await Console(agent.run_stream(task="What is the weather in Paris?"))
    await model_client.close()


# 5. Run the async main function
await main()

---------- TextMessage (user) ----------
What is the weather in Paris?
---------- ToolCallRequestEvent (weather_agent) ----------
[FunctionCall(id='call_bsHe5DMY2hzzwETdl61aO5sI', arguments='{"city":"Paris"}', name='get_weather')]
---------- ToolCallExecutionEvent (weather_agent) ----------
[FunctionExecutionResult(content='📍 Paris: Broken clouds, 22.78°C', name='get_weather', call_id='call_bsHe5DMY2hzzwETdl61aO5sI', is_error=False)]
---------- ModelClientStreamingChunkEvent (weather_agent) ----------
The current weather in Paris is partly cloudy with a temperature of 22.78°C.
